In [1]:
import sys
principal_path = '../'
if principal_path not in sys.path:
    sys.path.append('../')

In [2]:
import os
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import MNIST
import pytorch_lightning as pl
import torchmetrics
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from pytorch_lightning.loggers import WandbLogger
from lightning.pytorch.loggers import CSVLogger
from pytorch_lightning.loggers import TensorBoardLogger

from models.MNISTModel import MNISTModel
from models.MNISTModelWithBottelNeck import MNISTModelWithBottelNeck

In [3]:
SEED = 2024
pl.seed_everything(SEED)

Global seed set to 2024


2024

In [14]:
LEARING_RATE = 1e-3
BATCH_SIZE = 64
EPOCHS = 5
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
FOLDER_CHEKPOINTS = 'checkpoints'
FOLDER_ACTIVATIONS = 'activations'
print(f'Using {DEVICE}')

Using cuda


## Loggers

In [5]:
def get_number_of_parameters(model):
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    unit_scale = {
        '': 1,
        'K': 10 ** 3,
        'M': 10 ** 6,
        'B': 10 ** 9
    }
    for unit, scale in sorted(unit_scale.items(), key=lambda x: x[1], reverse=True):
        if num_params >= scale:
            return f'{round(num_params / scale, 1)}{unit}'
    return str(num_params)

def log_csv(model, model_name, folder = 'csv_logs'):
    number_of_parameters = get_number_of_parameters(model)
    return CSVLogger(
        save_dir=folder,
        name=f"{model_name}_{number_of_parameters}",
    )

def log_tensorboard(model, model_name, folder = 'tb_logs'):
    number_of_parameters = get_number_of_parameters(model)
    return TensorBoardLogger(
        save_dir=folder,
        name=f"{model_name}_{number_of_parameters}",
    )

def log_wandb(model, model_name, folder = 'wandb_logs'):
    number_of_parameters = get_number_of_parameters(model)
    return WandbLogger(
        save_dir=folder,
        name=f"{model_name}_{number_of_parameters}",
    )

## Download Dataset

In [6]:
def load_data(batch_size=BATCH_SIZE, num_workers=4):
    # Transformaciones para los datos
    transform = transforms.ToTensor()

    # Carga de datos de entrenamiento
    mnist_train = MNIST(os.getcwd(), train=True, download=True, transform=transform)
    
    # División entre entrenamiento y validación
    train_size = int(0.8 * len(mnist_train))
    val_size = len(mnist_train) - train_size
    mnist_train, mnist_val = random_split(mnist_train, [train_size, val_size])

    # DataLoader para entrenamiento y validación
    train_loader = DataLoader(mnist_train, batch_size=batch_size, num_workers=num_workers, shuffle=True, persistent_workers=True)
    val_loader = DataLoader(mnist_val, batch_size=batch_size, num_workers=num_workers, shuffle=False, persistent_workers=True)

    # Carga de datos de test
    mnist_test = MNIST(os.getcwd(), train=False, download=True, transform=transform)
    test_loader = DataLoader(mnist_test, batch_size=batch_size, num_workers=num_workers, persistent_workers=True)

    return train_loader, val_loader, test_loader

train_loader, val_loader, test_loader = load_data()

## Training

### MNISTmodel

In [7]:
model = MNISTModel(lr=LEARING_RATE)

In [10]:
loggers = [
    log_tensorboard(model, 'MLP'),
    log_csv(model, 'MLP'),
]

In [11]:
trainer = pl.Trainer(max_epochs=EPOCHS, logger=loggers)
trainer.fit(model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                | Params
--------------------------------------------------
0 | train_acc | MulticlassAccuracy  | 0     
1 | val_acc   | MulticlassAccuracy  | 0     
2 | test_acc  | MulticlassAccuracy  | 0     
3 | precision | MulticlassPrecision | 0     
4 | recall    | MulticlassRecall    | 0     
5 | f1        | MulticlassF1Score   | 0     
6 | layer_1   | Linear              | 100 K 
7 | layer_2   | Lin

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [12]:
trainer.test(model, test_loader)

You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│            f1             │    0.9716091752052307     │
│         precision         │    0.9743669033050537     │
│          recall           │    0.9738191366195679     │
│         test_acc          │    0.9757000207901001     │
│         test_loss         │    0.08198564499616623    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.08198564499616623,
  'test_acc': 0.9757000207901001,
  'precision': 0.9743669033050537,
  'recall': 0.9738191366195679,
  'f1': 0.9716091752052307}]

In [15]:
## save checkpoint
file = os.path.join(FOLDER_CHEKPOINTS, 'mnist_model.ckpt')
trainer.save_checkpoint(file)

### Bottleneck

In [16]:
bottel_neck_model = MNISTModelWithBottelNeck(lr=LEARING_RATE)

In [17]:
loggers = [
    log_tensorboard(bottel_neck_model, 'BottelNeck'),
    log_csv(bottel_neck_model, 'BottelNeck'),
]

In [18]:
trainer = pl.Trainer(max_epochs=EPOCHS, logger=log_tensorboard(bottel_neck_model, 'BottelNeck'))
trainer.fit(bottel_neck_model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: tb_logs\BottelNeck_269.4K


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                | Params
--------------------------------------------------
0 | train_acc | MulticlassAccuracy  | 0     
1 | val_acc   | MulticlassAccuracy  | 0     
2 | test_acc  | MulticlassAccuracy  | 0     
3 | precision | MulticlassPrecision | 0     
4 | recall    | MulticlassRecall    | 0     
5 | f1        | MulticlassF1Score   | 0     
6 | layer_1   | Linear              | 200 K 
7 | layer_2   | Linear              | 32.9 K
8 | layer_3   | Linear              | 33.0 K
9 | layer_4   | Linear              | 2.6 K 
--------------------------------------------------
269 K     Trainable params
0         Non-trainable params
269 K     Total params
1.078     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [19]:
trainer.test(bottel_neck_model, test_loader)

You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│            f1             │    0.9631451368331909     │
│         precision         │    0.9674546718597412     │
│          recall           │     0.965537965297699     │
│         test_acc          │    0.9678000211715698     │
│         test_loss         │    0.11508970707654953    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.11508970707654953,
  'test_acc': 0.9678000211715698,
  'precision': 0.9674546718597412,
  'recall': 0.965537965297699,
  'f1': 0.9631451368331909}]

In [20]:
## save checkpoint
file = os.path.join(FOLDER_CHEKPOINTS, 'mnist_model_bottel_neck.ckpt')
trainer.save_checkpoint(file)

## Activations

In [24]:
def load_data_for_activations():
    # load only one image of the test set
    transform = transforms.ToTensor()
    mnist_test = MNIST(os.getcwd(), train=False, download=True, transform=transform)
    test_loader = DataLoader(mnist_test, batch_size=1, shuffle=False)
    return test_loader

test_loader_act = load_data_for_activations()

In [21]:
minst_model_file = os.path.join(FOLDER_CHEKPOINTS, 'mnist_model.ckpt')
model_for_act = MNISTModel.load_from_checkpoint(minst_model_file)
model_for_act.eval()

MNISTModel(
  (train_acc): MulticlassAccuracy()
  (val_acc): MulticlassAccuracy()
  (test_acc): MulticlassAccuracy()
  (precision): MulticlassPrecision()
  (recall): MulticlassRecall()
  (f1): MulticlassF1Score()
  (layer_1): Linear(in_features=784, out_features=128, bias=True)
  (layer_2): Linear(in_features=128, out_features=256, bias=True)
  (layer_3): Linear(in_features=256, out_features=10, bias=True)
)

In [26]:
def create_neuron_dataframe(layer_neurons, num_rows, test_loader_act, model_for_act):
    # Creación de las columnas para cada neurona
    columns = ['Number']
    for layer_index, neurons in enumerate(layer_neurons):
        columns += [f'Layer{layer_index+1}_Neuron{i+1}' for i in range(neurons)]

    # Pre-creación del DataFrame
    df = pd.DataFrame(index=range(num_rows), columns=columns)

    # Llenar el DataFrame
    for idx, batch in enumerate(tqdm(test_loader_act, desc='Running activations')):
        x, y = batch
        _, r = model_for_act(x, record_activations=True)

        # Construir fila para el DataFrame
        row = {'Number': y.item()}
        for layer_index, neurons in enumerate(layer_neurons):
            layer_activation = r[layer_index].cpu().detach().numpy()[0]
            for neuron_index in range(neurons):
                row[f'Layer{layer_index+1}_Neuron{neuron_index+1}'] = layer_activation[neuron_index]

        df.loc[idx] = row
    
    return df

In [27]:
# Ejemplo de uso
layer_neurons = [128, 256, 10]  # Lista con el número de neuronas en cada capa
num_rows = len(test_loader_act)  # Número de filas en el DataFrame
df = create_neuron_dataframe(layer_neurons, num_rows, test_loader_act, model_for_act)

Running activations: 100%|██████████| 10000/10000 [00:07<00:00, 1256.41it/s]


In [29]:
df.head()

,Number,Layer1_Neuron1,Layer1_Neuron2,Layer1_Neuron3,Layer1_Neuron4,Layer1_Neuron5,Layer1_Neuron6,Layer1_Neuron7,Layer1_Neuron8,Layer1_Neuron9,...,Layer3_Neuron1,Layer3_Neuron2,Layer3_Neuron3,Layer3_Neuron4,Layer3_Neuron5,Layer3_Neuron6,Layer3_Neuron7,Layer3_Neuron8,Layer3_Neuron9,Layer3_Neuron10
0,7.0,0.0,0.0,0.0,0.399097,0.0,0.0,0.0,0.0,0.0,...,-22.055817,-17.074514,-12.136678,-11.195572,-25.313034,-18.126488,-34.18375,-0.000019,-16.948915,-15.565269
1,2.0,0.0,0.0,3.122375,2.949144,2.404661,1.715258,0.338522,0.128126,1.610624,...,-15.90349,-8.095853,-0.000322,-11.127125,-27.803934,-17.198702,-14.273718,-19.698458,-13.450046,-30.231627
2,1.0,1.827291,1.748518,0.699948,0.539016,0.0,0.628443,0.588888,0.0,0.020213,...,-16.210146,-0.000385,-11.081397,-12.736809,-10.941203,-11.521659,-11.719425,-8.667583,-8.749459,-14.620137
3,0.0,0.0,0.0,0.614674,0.0,1.043494,0.438858,0.0,3.829711,2.498251,...,-0.000426,-13.427382,-9.176719,-14.58223,-15.579443,-13.185836,-8.165977,-12.736848,-13.24618,-10.434205
4,4.0,0.0,0.0,1.187802,0.092359,0.0,0.0,0.0,2.307845,1.650951,...,-13.418967,-15.270522,-12.993325,-18.622927,-0.005147,-10.95846,-15.022395,-10.377352,-9.788157,-5.293344


In [30]:
# save activations csv
df.to_csv('activations_minist_model.csv', index=False)